# Project Part 1
**Eliya Zaguri ID 207313131**

[GitHub Repository](https://github.com/eliyazaguri/car-price-prediction-project)

In [11]:
# שלב 1: יבוא הספריות הנדרשות
import requests
from bs4 import BeautifulSoup
import pandas as pd

# שלב 2: פונקציה לקבלת תוכן ה-HTML של הדף
def get_html(url):
    response = requests.get(url)
    if response.status_code == 200:
        response.encoding = 'utf-8'  # Ensure correct encoding
        return response.text
    else:
        return None

# שלב 3: פונקציה לאיסוף הנתונים מהדף הראשי
def scrape_main_page_data(card):
    data = {}
    data['יצרן'] = 'פיאט'  # אנו מניחים שהיצרן הוא תמיד פיאט במודעה זו
    title = card.find('h2', class_='card-title').text.strip() if card.find('h2', class_='card-title') else 'N/A'
    if title.startswith('פיאט'):
        data['דגם'] = title.replace('פיאט', '').strip()
    else:
        data['דגם'] = 'N/A'
    data['מחיר'] = card.find('div', class_='price').text.strip() if card.find('div', class_='price') else 'N/A'
    data['אזור'] = card.find('span', class_='region').text.strip() if card.find('span', class_='region') else 'N/A'
    data['עיר'] = card.find('span', class_='city').text.strip() if card.find('span', 'city') else 'N/A'
    return data

# שלב 4: פונקציה לאיסוף הנתונים מהטבלה בדף המודעה
def scrape_car_data(car_soup, main_page_data):
    data = main_page_data.copy()
    table = car_soup.find('table', {'class': 'table table-sm mb-4'})
    if not table:
        print("No table found on page.")
        return data
    
    rows = table.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        if len(cols) == 2:
            key = cols[0].text.strip()
            value = cols[1].text.strip()
            data[key] = value

    # איסוף מידע נוסף לפי הדרישות
    info_div = car_soup.find('div', class_='d-flex flex-row align-items-center justify-content-center flex-wrap')
    if info_div:
        creation_date_div = info_div.find_all('div', class_='px-3')
        if len(creation_date_div) >= 2:
            creation_date = creation_date_div[0].text.strip().split(': ')[-1]
            repub_date = creation_date_div[1].text.strip().split(': ')[-1]
            data['תאריך יצירה'] = pd.to_datetime(creation_date, dayfirst=True).strftime('%Y-%m-%d')
            data['תאריך הקפצה'] = pd.to_datetime(repub_date, dayfirst=True).strftime('%Y-%m-%d')
        else:
            data['תאריך יצירה'] = 'N/A'
            data['תאריך הקפצה'] = 'N/A'
    else:
        data['תאריך יצירה'] = 'N/A'
        data['תאריך הקפצה'] = 'N/A'
    
    description = car_soup.find('p', class_='text-word-break')
    if description:
        data['תיאור'] = description.text.strip()
    else:
        data['תיאור'] = 'N/A'
    
    # מציאת מספר התמונות
    images_wrapper = car_soup.find('div', class_='swiper-wrapper')
    if images_wrapper:
        image_slides = images_wrapper.find_all('div', class_='swiper-slide')
        data['מספר תמונות'] = len(image_slides)
    else:
        data['מספר תמונות'] = 0

    return data

# שלב 5: פונקציה לאיסוף הקישורים למודעות מהרשימה הראשית
def get_car_links(url):
    html = get_html(url)
    if not html:
        return []

    soup = BeautifulSoup(html, 'html.parser')
    links = []
    main_page_data_list = []

    for card in soup.find_all('div', class_='card-body p-md-3'):
        a = card.find('a', href=True)
        if a:
            href = a['href']
            if href.startswith('/ad/'):
                full_link = 'https://www.ad.co.il' + href
                if 'פיאט' in card.text:
                    links.append(full_link)
                    main_page_data_list.append(scrape_main_page_data(card))

    return links, main_page_data_list

# שלב 6: פונקציה מרכזית לאיסוף הנתונים מכל המודעות בדף
def scrape_all_cars(start_url):
    car_links, main_page_data_list = get_car_links(start_url)
    print(f"Found {len(car_links)} car links.")
    
    car_data = []

    for link, main_page_data in zip(car_links, main_page_data_list):
        print(f"Scraping data from {link}")
        car_html = get_html(link)
        if not car_html:
            print(f"Failed to retrieve {link}")
            continue

        car_soup = BeautifulSoup(car_html, 'html.parser')
        data = scrape_car_data(car_soup, main_page_data)
        if data:
            car_data.append(data)

    return car_data

# שלב 7: פונקציה לניקוי תיאורים ותיקון תווים
def clean_description(desc):
    if pd.isna(desc):
        return desc
    return ' '.join(desc.split())

# שלב 8: דוגמה לשימוש בפונקציה לאיסוף הנתונים ושמירתם
start_url = 'https://www.ad.co.il/car?sp261=13902'
car_data = scrape_all_cars(start_url)

# המרת הנתונים לפורמט DataFrame של pandas
df = pd.DataFrame(car_data)

# שלב 9: הגדרת כל העמודות הנדרשות, והוספת עמודות חסרות במידת הצורך
required_columns = [
    'יצרן', 'דגם', 'מחיר', 'אזור', 'עיר', 'שנה', 'יד', 'ת. הילוכים', 'נפח', 'סוג מנוע', 
    'בעלות קודמת', 'בעלות נוכחית', 'טסט עד', 'ק"מ', 'צבע', 'תיאור', 'מספר תמונות', 'תאריך יצירה', 'תאריך הקפצה'
]
for col in required_columns:
    if col not in df.columns:
        df[col] = 'N/A'

# אם יש עמודות כפולות של "טסט" ו-"טסט עד", נוודא שמאחדים אותן
if 'טסט עד' in df.columns and 'טסט' in df.columns:
    df['טסט עד'] = df['טסט עד'].combine_first(df['טסט'])
    df = df.drop(columns=['טסט'])

# ניקוי כל העמודות מתווים לא תקינים ורווחים מיותרים
for col in df.columns:
    df[col] = df[col].apply(lambda x: x.strip() if isinstance(x, str) else x)

# ניקוי התיאור
df['תיאור'] = df['תיאור'].apply(clean_description)

# שלב 10: שמירת הנתונים המעודכנים לקובץ CSV חדש
df.to_csv('cleaned_car_data.csv', index=False, encoding='utf-8-sig')

print("Data scraped and saved to cleaned_car_data.csv")


Found 48 car links.
Scraping data from https://www.ad.co.il/ad/16183484
Scraping data from https://www.ad.co.il/ad/16093061
Scraping data from https://www.ad.co.il/ad/16053968
Scraping data from https://www.ad.co.il/ad/16053457
Scraping data from https://www.ad.co.il/ad/16040471
Scraping data from https://www.ad.co.il/ad/15943965
Scraping data from https://www.ad.co.il/ad/15512048
Scraping data from https://www.ad.co.il/ad/15781247
Scraping data from https://www.ad.co.il/ad/15512138
Scraping data from https://www.ad.co.il/ad/15742382
Scraping data from https://www.ad.co.il/ad/15371056
Scraping data from https://www.ad.co.il/ad/14928292
Scraping data from https://www.ad.co.il/ad/14760739
Scraping data from https://www.ad.co.il/ad/14668072
Scraping data from https://www.ad.co.il/ad/14655334
Scraping data from https://www.ad.co.il/ad/14651769
Scraping data from https://www.ad.co.il/ad/14611346
Scraping data from https://www.ad.co.il/ad/14599028
Scraping data from https://www.ad.co.il/ad/1